In [11]:
##
#Load Packages
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import sys
import nibabel as nib

In [12]:
##
#Define Path to Code database
DirPath = '/Your/Path/To/Code/'

#Define Output Path for Diffusion Volumes
OutputPath = 'Your/Output/Path/'

#Define Path to HCP1065 MD Map (in FSL this is equivalent to $FSLDIR/data/standard/FSL_HCP1065_MD_1mm.nii.gz)
MDPath = '/usr/local/fsl/data/standard/FSL_HCP1065_MD_1mm.nii.gz'

In [13]:
##
#Load functions
sys.path.append(''.join([DirPath,'bin']))
from EPGMotion import *
from ParameterOptionsSimulation import *
from MotionSimulation import *

In [14]:
##
#Read Parameters
opt=ParameterOptionsSimulation()

In [15]:
##
#Conventional EPG simulates a maximum k-value pathway equal to the number of TRs. This piece of code identifies where increasing the k-value leads to no meaningful change in the signal, with subsequent thresholding to accelerate modelling
opt["kOrder"] = kOrder(opt)

In [16]:
##
#Load data

##
#Define slice of MD map to sample (you can process an entire brain, but a single slice is faster).
slice = 80

#Load diffusion coefficient map (HCP1065 MD map) & Affine
Dataset = np.atleast_3d(nib.load(MDPath).get_fdata())
Affine = nib.load(MDPath).affine

##
#Pull slice
D = Dataset[:,:,slice][:,:,np.newaxis]

##
#Define location of the slice relative to the centre of the brain (here we assume the brain centre is in the middle of the FOV)
Offset = round(Dataset.shape[-1]/2)-slice

##
#Get Voxel Dimensions (in mm)
VoxDims = nib.affines.voxel_sizes(Affine)

##
#Create Brain Mask 
Mask = D > 0

##
#Obtain mask indices
MaskIdx = np.where(Mask==1)

##
#Vectorise Diffusion & Mask map for dictionary to accelerate EPG fitting
opt['D'] = np.asarray([*D[Mask==1]], dtype='f8')
opt['Mask'] = np.asarray([*Mask[Mask==1]], dtype='f8')

In [17]:
##

##
#Define T1 and T2 values as equal across the simulation
opt['T1'] = opt['Mask']*opt['T1']
opt['T2'] = opt['Mask']*opt['T2']

##
#Define B1 as constant across the image (i.e. no B1 inhomogeneity)
opt['B1'] = np.asarray([*Mask[Mask==1]], dtype='f8') 

In [18]:
##
#Define Motion Operators for each simulation - Translation Operator (mm/s), Rotation Operator (rad/s) & Cardiac Operator (vector defines maximum beat velocity in mm/s)

##
#Define consistent motion per TR 
RandMotion=False
#Define Heart Rate for Cardiac Operator (beats/min)
HeartRate = 50

##
#Define Operators
Translation= np.array([0,0,0])
Rotation = np.array([0,0,0])
Cardiac = np.array([0,0,0.4])

In [19]:
##
#Simulation - No Motion

##
#Initialise Output Array
F = np.zeros((*Mask.shape,opt['nTR'].astype(np.int32)[0]), dtype='c8')

##
#Perform Forward Simulation
F[MaskIdx[0],MaskIdx[1],MaskIdx[2],:] = EPGMotionWholeImage(opt)

##
#Save Data
nib.save(nib.Nifti1Image(np.abs(F)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, 'Magnitude_noMotion.nii.gz']))
nib.save(nib.Nifti1Image(np.angle(F*-1)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, 'Phase_noMotion.nii.gz']))
nib.save(nib.Nifti1Image(F*0,Affine),''.join([OutputPath, 'T_noMotion.nii.gz']))

In [20]:
##
#Simulation - Cardiac Motion

##
#Initialise Output Array
FCardiac = np.zeros((*Mask.shape,opt['nTR'].astype(np.int32)[0]), dtype='c8')

##
#Establish Velocity Timeseries Profile
T_Cardiac = MotionOperator(opt,Translation,Rotation,Cardiac,Mask,VoxDims,HeartRate=HeartRate,offset=Offset)

##
#Peform Forward Simulation
FCardiac[MaskIdx[0],MaskIdx[1],MaskIdx[2],:] = EPGMotionWholeImage(opt,T_Cardiac[Mask==1,:])

##
#Save Data
nib.save(nib.Nifti1Image(np.abs(FCardiac)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, 'Magnitude.nii.gz']))
nib.save(nib.Nifti1Image(np.angle(FCardiac*-1)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/Phase.nii.gz']))
nib.save(nib.Nifti1Image(T_Cardiac*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, 'T.nii.gz']))